In [ ]:
# import sys
# sys.path.append('../python_scripts')

import matplotlib.pyplot as plt
import pylab as p
import numpy as np
import os
from glob import glob

import sys
sys.path.insert(0, '/data/disk1/fragkos/repos/CE_mesa/python_scripts')
import mesa as ms
import binary as bn


Lsun = 3.828e33
secyer = 24.0*3600.0*365.24
GGG = 6.674e-8
Msun = 1.989e33
Rsun = 6.955e10
clight = 29979245800
pi = 3.1415
crad = 7.57e-15



%matplotlib inline 
plt.rcParams['figure.figsize'] = 12, 6  # that's default image size for this interactive session

run_path= "/data/disk1/fragkos/mesa_projects/CE/CE_paperI_BBH/"
#run_path= "/data/disk1/fragkos/mesa_projects/CE/DVU_Mexico/1.4M_15M_700R/"


run_names = [d for d in os.listdir(run_path) if os.path.isdir(os.path.join(run_path, d))]

run_names = ["step2_macleod_NoMLT"]
print(run_names)

In [ ]:
data_path = run_path+run_names[0]+"/LOGS/"
a = ms.mesa(data_path=data_path, parallel=True, abundances=False, log_abundances = True, Yaxis='log_mass', Xaxis="log_inv_star_age",czones=False, Variable='v_div_vesc', orbit=True)
print len(a.profiles)



In [ ]:
i=len(a.profiles)-1



Lsun = 3.828e33
secyer = 24.0*3600.0*365.24
GGG = 6.674e-8
Msun = 1.989e33
Rsun = 6.955e10
clight = 29979245800
pi = 3.1415
crad = 7.57e-15

R_companion = 38.843286625899026 #Rsun
M_bh = 8.0 #Msun




def roche_lobe(m1, m2):
    q_mass1 = m1 / m2
    q_mass3 = q_mass1 ** (1.0 / 3.0)
    q_mass2 = q_mass1 ** (2.0 / 3.0)
    lobe = (0.49 * q_mass2) / (0.6 * q_mass2 + np.log(1.0 + q_mass3))
    return lobe  # Dimensionless. In units of orbital separation

Rcoord=10**a.profiles[i]["logR"]
Rrl=roche_lobe(a.profiles[i]["mass"],M_bh)*R_companion

plt.plot(a.profiles[i]["mass"],Rcoord,  color='k')
plt.plot(a.profiles[i]["mass"],Rrl,  color='b')


idx = np.argwhere(np.diff(np.sign(Rcoord - Rrl)) != 0).reshape(-1) + 0
plt.plot(a.profiles[i]["mass"][idx], Rcoord[idx], 'ro')

print(a.profiles[i]["mass"][idx][0], Rcoord[idx][0])

plt.xlabel(r'Mass Coordinate ($M_{\odot}$)', size=20)
plt.ylabel(r'Radius Coordinate ($R_{\odot}$)', size=20)
#plt.yscale('log')
plt.ylim(0, 200)
plt.show()

In [ ]:
# font = {'family' : 'normal',
#         'weight' : 'bold',
#         'size'   : 22}

# plt.rc('font', **font)

#plt.style.use('classic')

plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.serif'] = 'Ubuntu'
plt.rcParams['font.monospace'] = 'Ubuntu Mono'
plt.rcParams['font.size'] = 10
plt.rcParams['axes.labelsize'] = 10
plt.rcParams['axes.labelweight'] = 'normal'
plt.rcParams['axes.titlesize'] = 10
plt.rcParams['xtick.labelsize'] = 8
plt.rcParams['ytick.labelsize'] = 8
plt.rcParams['legend.fontsize'] = 10
plt.rcParams['figure.titlesize'] = 12



xlim_min = 5
xlim_max = np.max(a.history['star_age'])



fig, ax = plt.subplots(2,2, figsize=(8, 6))



ax[0,0].plot(a.history['star_age'], a.history["CE_companion_position_r"], linewidth=2, color='k', label = "CO position $r_{CO}$")
ax[0,0].plot(a.history['star_age'], 10**a.history["log_R"], color='r', label="Stellar radius")
ax[0,0].plot(a.history['star_age'], a.history["tau10_radius"], color='r', linestyle="--", label=r"Radius @ $\tau$ = 10")
ax[0,0].plot(a.history['star_age'], a.history["tau100_radius"], color='r',linestyle="-.", label=r"Radius @ $\tau$ = 100")

ax[0,0].legend(fontsize=8, bbox_to_anchor=(0.9, 0.83),
           bbox_transform=plt.gcf().transFigure)

ax[0,0].set_xlabel("Time (yr)")
ax[0,0].set_ylabel(r"Radius $(R_{\odot})$")
#ax[0,0].set_xlim(0.1, 60)
#plt.ylim(200, 350)
# plt.xscale('log')
ax[0,0].set_yscale('log')
#ax[0,0].set_xlim(0,20)





ax[0,1].plot(np.log10(a.history['star_age']), np.log10(a.history["CE_companion_position_r"]), linewidth=2, color='k')
ax[0,1].set_xlabel("log(Time [yr])")
ax[0,1].set_ylabel(r"log($R_{CO}\ [R_{\odot}]$)")
ax[0,1].set_xlim(np.log10(xlim_min), np.log10(xlim_max))
# ax[0,1].set_ylim(1.0, 1.5)




idx = np.where(a.history['star_age'] > xlim_min)
z = np.polyfit(np.log10(a.history['star_age'][idx]), np.log10(a.history["CE_companion_position_r"][idx]), 1)
print(z)
ax[0,1].plot(np.log10(a.history['star_age'][idx]), z[1]+z[0]*np.log10(a.history['star_age'][idx]), linewidth=2, color='b',linestyle=":",label=r"$r_{CO}/R_{\odot} \simeq "+str(round(10.**z[1], 1))+" (t/yr)^{"+str(round(z[0], 2))+"}$")
ax[0,1].legend(loc=0, fontsize=8)



ax[1,0].plot(a.history['star_age'], a.history["CE_companion_position_m"], linewidth=2, color='k', label = r"CO position $m_{CO}$")
ax[1,0].plot(a.history['star_age'], a.history["star_mass"], color='r', label="Stellar mass")
ax[1,0].plot(a.history['star_age'], a.history["tau10_mass"], color='r', linestyle="--", label=r"Mass coord. @ $\tau$ = 10")
ax[1,0].plot(a.history['star_age'], a.history["tau100_mass"], color='r',linestyle="-.", label=r"Mass coord. @ $\tau$ = 100")

ax[1,0].legend(loc=0, fontsize=8)

ax[1,0].set_xlabel(r"Time (yr)")
ax[1,0].set_ylabel(r"Mass $(M_{\odot})$")
ax[1,0].set_xlim(0,20)


#plt.xlim(0.1, 60)
#plt.ylim(200, 350)
# plt.xscale('log')
# plt.yscale('log')





ax[1,1].plot(np.log10(a.history['star_age']), np.log10(a.history["CE_companion_position_m"]), linewidth=2, color='k')
ax[1,1].set_xlabel(r"log(Time [yr])")
ax[1,1].set_ylabel(r"log($ m_{CO}\ [M_{\odot}])$")
ax[1,1].set_xlim(np.log10(xlim_min), np.log10(xlim_max))
#ax[1,1].set_ylim(0.62, 0.64)

idx = np.where(a.history['star_age'] > xlim_min)
z = np.polyfit(np.log10(a.history['star_age'][idx]), np.log10(a.history["CE_companion_position_m"][idx]), 1)
print(z)
ax[1,1].plot(np.log10(a.history['star_age'][idx]), z[1]+z[0]*np.log10(a.history['star_age'][idx]), color='b',linestyle=":",label=r"$m_{CO}/M_{\odot} \simeq "+str(round(10.**z[1], 1))+" (t/yr)^{"+str(round(z[0], 2))+"}$")
ax[1,1].legend(loc=0, fontsize=8)




plt.tight_layout()
plt.savefig("inspiral_R_M_vs_t_BH.png", dpi=300)
plt.show()


In [ ]:
fig, ax = plt.subplots(1,3, figsize=(12, 3))

idx = np.where(a.history["model_number"] > 100)

ax[2].plot(a.history['log_Teff'][idx], a.history["log_L"][idx], color='k',linewidth=2)
xwidth = max(a.history['log_Teff']) - min(a.history['log_Teff'])
#plt.xlim(0.1*xwidth + max(a.history['log_Teff']), -0.1*xwidth + min(a.history['log_Teff']))
ax[2].set_xlabel(r"${\rm log}({\rm T}_{\rm eff}\ [K])$")
ax[2].set_ylabel(r"${\rm log}(L\ [L_{\odot}])$")
ax[2].plot(a.history['log_Teff'][idx[0][0]], a.history["log_L"][idx[0][0]],"*", color="b", markersize=10)
ax[2].invert_xaxis()


sigma_B = 5.6704*10**(-5)
Rsun = 6.957e10
Lsun = 3.839e33
Msun = 1.989e33

radii = np.asarray([100,200,500,1000,2000,5000,10000,20000,50000,100000])
Teff = np.linspace(1000,4000,100)
for radius in radii:
    ax[2].plot(np.log10(Teff),np.log10((4.*3.1415*(radius*Rsun)**2*sigma_B*Teff**4)/Lsun),linestyle=":",color="k")


# aa=1
# bb=4
# MM =16.
# AA = 1./101.
# BB = 7./51.
# CC = 13./51.
# DD = np.log10(4000)
# logL_hayashi = np.linspace(np.min(a.history["log_L"]), np.max(a.history["log_L"]), num=20)
# logTeff_hayashi = AA*logL_hayashi + BB* np.log10(MM) + CC*np.log10(0.5) + DD

# plt.plot(logTeff_hayashi, logL_hayashi, color='g')
# print(logTeff_hayashi)
# print(logL_hayashi)
# plt.show()

ax[0].plot(a.history['star_age'][idx], a.history["log_L"][idx], color='k',linewidth=2)
ax[0].set_xlabel("Star Age (yr)")
ax[0].set_ylabel(r"${\rm log}(L\ [L_{\odot}])$")
ax[0].set_xlim(0,20)




ax[1].plot(a.history['star_age'][idx], 10**a.history["log_Teff"][idx], color='k',linewidth=2)
ax[1].set_xlabel("Star Age (yr)")
ax[1].set_ylabel(r'$T_{\rm eff}\ (K)$')
ax[1].set_xlim(0,20)



# fig = plt.figure(figsize=(4,3), dpi=400)
# plt.plot(a.history['star_age'], a.history["u_mag"], label="U")
# plt.plot(a.history['star_age'], a.history["b_mag"], label="B")
# plt.plot(a.history['star_age'], a.history["v_mag"], label="V")
# plt.plot(a.history['star_age'], a.history["r_mag"], label="R")
# plt.plot(a.history['star_age'], a.history["i_mag"], label="I")


# plt.xlabel("Star Age (yr)")
# plt.ylabel(r'Abs. magnitude$')

plt.tight_layout()
plt.savefig("L_Teff_NS.png",dpi=300)
plt.show()




In [ ]:
#LOAD BH models

run_path= "/data/disk1/fragkos/mesa_projects/CE/DVU_Mexico/8M_30M_1400R/"


run_names = ["step2_macleod_setE2_noMLT", "step2_lee_setE2_noMLT", "step2_macleod_noMLT", "step2_macleod_setE2", "step2_macleod_setE2_noMLT_NoRot", "step2_macleod_setE2_noMLT_NoSync"]
labels=["standard",r"F_${drag}$ from Lee & Stahler", "constant spec. E/AM", "with convection","no rotation", "no initial sync."]


modelsBH = []

for run_name in run_names:
    data_path = run_path+run_name+"/LOGS/"
    b = ms.mesa(data_path=data_path, parallel=True, abundances=False, log_abundances = True, Yaxis='log_mass', Xaxis="log_inv_star_age",czones=False, Variable='v_div_vesc', orbit=True)
    modelsBH.append(b) 

#LOAD NS models

# run_path= "/data/disk1/fragkos/mesa_projects/CE/DVU_Mexico/1.4M_15M_700R/"


# run_names = ["step2_macleod_setE2_NoMLT", "step2_lee_setE2_NoMLT", "step2_macleod_NoMLT", "step2_macleod_setE2", "step2_macleod_NoMLT_NoRot", "step2_macleod_setE2_NoMLT_NoSync"]
# labels=["standard",r"F_${drag}$ from Lee & Stahler", "constant spec. E/AM", "with convection","no rotation", "no initial sync."]



# modelsNS = []

# for run_name in run_names:
#     data_path = run_path+run_name+"/LOGS/"
#     b = ms.mesa(data_path=data_path, parallel=True, abundances=False, log_abundances = True, Yaxis='log_mass', Xaxis="log_inv_star_age",czones=False, Variable='v_div_vesc', orbit=True)
#     modelsNS.append(b) 



In [ ]:
fig = plt.figure(figsize=(4,3), dpi=300)

static_t = np.load("int_time_BH.npy")
static_r = np.load("int_r_BH.npy")
static_m = np.load("int_m_BH.npy")

plt.xlabel("log(Time [yr])")
plt.ylabel(r"log($r_{CO}\ [R_{\odot}]$)")

plt.xlabel("log(Time [yr])")
plt.ylabel(r"log($r_{CO}\ [R_{\odot}]$)")
plt.xlim(0, 30)
plt.ylim(1.5, 3.3)


i=0
for run_name in run_names:
    plt.plot(modelsBH[i].history['star_age'], np.log10(modelsBH[i].history["CE_companion_position_r"]), linewidth=2, label=labels[i])
    i=i+1
plt.plot(static_t,np.log10(static_r),linewidth=2,linestyle="--",color="k")
plt.legend(loc=0,fontsize=8)
plt.tight_layout()
plt.savefig("inspiral_R_vs_time_comparison_BH.pdf")
plt.show()

fig = plt.figure(figsize=(4,3), dpi=300)

plt.xlabel("log(Time [yr])")
plt.ylabel(r"log($m_{CO}\ [R_{\odot}]$)")

plt.xlabel("log(Time [yr])")
plt.ylabel(r"$m_{CO}\ [R_{\odot}]$")
plt.xlim(0, 30)
plt.ylim(12,28)


i=0
for run_name in run_names:
    plt.plot(modelsBH[i].history['star_age'], modelsBH[i].history["CE_companion_position_m"], linewidth=2, label=labels[i])
    i=i+1
plt.plot(static_t,static_m,linewidth=2,linestyle="--",color="k")

plt.legend(loc=0,fontsize=8)
plt.tight_layout()
plt.savefig("inspiral_M_vs_time_comparison_BH.pdf")
plt.show()






fig = plt.figure(figsize=(4,3), dpi=300)

static_t = np.load("int_time_NS.npy")
static_r = np.load("int_r_NS.npy")
static_m = np.load("int_m_NS.npy")

plt.xlabel("log(Time [yr])")
plt.ylabel(r"log($r_{CO}\ [R_{\odot}]$)")

plt.xlabel("log(Time [yr])")
plt.ylabel(r"log($r_{CO}\ [R_{\odot}]$)")
plt.xlim(0, 25)
plt.ylim(1.0, 2.8)


i=0
for run_name in run_names:
    plt.plot(modelsNS[i].history['star_age'], np.log10(modelsNS[i].history["CE_companion_position_r"]), linewidth=2, label=labels[i])
    i=i+1
plt.plot(static_t,np.log10(static_r),linewidth=2,linestyle="--",color="k")
plt.legend(loc=0,fontsize=8)
plt.tight_layout()
plt.savefig("inspiral_R_vs_time_comparison_NS.pdf")
plt.show()

fig = plt.figure(figsize=(4,3), dpi=300)

plt.xlabel("log(Time [yr])")
plt.ylabel(r"log($m_{CO}\ [R_{\odot}]$)")

plt.xlabel("log(Time [yr])")
plt.ylabel(r"$m_{CO}\ [R_{\odot}]$")
plt.xlim(0, 25)
plt.ylim(0,15)


i=0
for run_name in run_names:
    plt.plot(modelsNS[i].history['star_age'], modelsNS[i].history["CE_companion_position_m"], linewidth=2, label=labels[i])
    i=i+1
plt.plot(static_t,static_m,linewidth=2,linestyle="--",color="k")

plt.legend(loc=0,fontsize=8)
plt.tight_layout()
plt.savefig("inspiral_M_vs_time_comparison_NS.pdf")
plt.show()





In [ ]:
companion_position_m_at_profiles = np.interp(a.profile_age, a.history['star_age'],a.history["CE_companion_position_m"])
companion_position_r_at_profiles = np.interp(a.profile_age, a.history['star_age'],a.history["CE_companion_position_r"])


gamma3_at_companion = np.zeros(len(a.profile_age))
omega_div_omega_kep_at_companion = np.zeros(len(a.profile_age))
omega_div_omega_crit_at_companion = np.zeros(len(a.profile_age))
v_rel_div_csound = np.zeros(len(a.profile_age))
v_rel = np.zeros(len(a.profile_age))


for i in range(len(a.profile_age)):
    gamma3_at_companion[i] = np.interp(companion_position_r_at_profiles[i],10**a.profiles[i]["logR"],a.profiles[i]["gamma3"])


fig = plt.figure(figsize=(4,3), dpi=400)

plt.xlabel(r"r$_{CO}$ [R$_\odot$]")
plt.ylabel(r"$\gamma_3$ @ CO position")
plt.ylim(1.,1.5)
plt.plot(companion_position_r_at_profiles, gamma3_at_companion, linewidth=2)
plt.gca().invert_xaxis()
plt.show()

In [ ]:
d_CE_companion_position_m_dt = -np.gradient(a.history["CE_companion_position_m"])/(10.**a.history["log_dt"])
d_CE_companion_position_r_dt = -np.gradient(a.history["CE_companion_position_r"])/(10.**a.history["log_dt"])


fig = plt.figure(figsize=(4,3), dpi=400)

plt.ylabel(r"Inspiral rate (M$_{\odot}$/yr)")
plt.xlabel(r"Radial coordinate of CO (R$_{\odot}$)")
# plt.ylim(1.,1.5)
plt.plot(a.history["CE_companion_position_r"], d_CE_companion_position_m_dt, linewidth=2)
plt.yscale("log")
plt.ylim(0.00001,100)
plt.gca().invert_xaxis()

plt.plot(a.history["CE_companion_position_r"],a.history["Mdot_macleod"]/Msun*secyer)

plt.tight_layout()

plt.savefig("inspiral_rate_m_vs_R_NS.png",dpi=300)


plt.show()


fig = plt.figure(figsize=(4,3), dpi=400)

plt.ylabel(r"Inspiral rate (M$_{\odot}$/yr)")
plt.xlabel(r"Time (yr)")
# plt.ylim(1.,1.5)
plt.plot(a.history["star_age"], d_CE_companion_position_m_dt, linewidth=2)
plt.yscale("log")
plt.ylim(0.00001,100)
plt.xlim(0,20)


plt.plot(a.history["star_age"],a.history["Mdot_macleod"]/Msun*secyer)

plt.tight_layout()

plt.savefig("inspiral_rate_m_vs_time_NS.png",dpi=300)

plt.show()

fig = plt.figure(figsize=(4,3), dpi=400)

plt.ylabel(r"Inspiral rate (R$_{\odot}$/yr)")
plt.xlabel(r"Radial coordinate of CO (R$_{\odot}$)")
# plt.ylim(1.,1.5)
plt.plot(a.history["CE_companion_position_r"], d_CE_companion_position_r_dt, linewidth=2)
plt.yscale("log")
plt.ylim(0.01,10000)
plt.gca().invert_xaxis()

#plt.plot(a.history["CE_companion_position_r"],a.history["Mdot_macleod"]/Msun*secyer)

plt.tight_layout()

plt.savefig("inspiral_rate_r_vs_R_NS.png",dpi=300)


plt.show()


fig = plt.figure(figsize=(4,3), dpi=400)

plt.ylabel(r"Inspiral rate (R$_{\odot}$/yr)")
plt.xlabel(r"Time (yr)")
# plt.ylim(1.,1.5)
plt.plot(a.history["star_age"], d_CE_companion_position_r_dt, linewidth=2)
plt.yscale("log")
plt.ylim(0.01,10000)
plt.xlim(0,20)


#plt.plot(a.history["star_age"],a.history["Mdot_macleod"]/Msun*secyer)

plt.tight_layout()

plt.savefig("inspiral_rate_r_vs_time_NS.png",dpi=300)

plt.show()

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(8, 3))



secyer = 31622400.0

lum = 10.0**a.history['log_L']
time_step = 10.0**a.history['log_dt'] * secyer
time = a.history["star_age"]

E_CE = np.cumsum(time_step*a.history["CE_energy_rate"])
ax[0].plot(time, E_CE, label="Injected")




E_rad = np.cumsum(lum*time_step)* 3.839e33
ax[0].plot(time, E_rad , label="Radiated away", linestyle='solid')

# E_nuc = np.cumsum(lum[0]*time_step)* 3.839e33
E_nuc = np.cumsum(10.**a.history["log_Lnuc"]*time_step)* 3.839e33

ax[0].plot(time, E_nuc , label="Nuclear", linestyle='solid')



E_kin = a.history["total_linear_kinetic_energy"]
ax[0].plot(time, a.history["total_linear_kinetic_energy"], linestyle='dashed', label=r"$\Delta E_{kinetic}$")
#ax[0].plot(time, a.history["total_rotational_kinetic_energy"], linestyle='dashed', label="Rotational")


E_grav = a.history["total_gravitational_energy"]-a.history["total_gravitational_energy"][0]
E_grav = E_grav -np.min(E_grav)
ax[0].plot(time, E_grav, linestyle='dashed', label=r"$\Delta E_{gravitational}$")

E_int = -(a.history["total_internal_energy"]-a.history["total_internal_energy"][0])
E_int = E_int -np.min(E_int)
#print(E_grav)
ax[0].plot(time, E_int, linestyle='dashed', label=r"$-\Delta E_{internal}$")


# ax[0].plot(time, E_CE+E_nuc+E_int-E_rad-E_kin-E_grav, linestyle='dashed')








Msun = 1.988e33
G = 6.6725e-8
Rsun = 6.957e10
E_available = G*a.history['he_core_mass'][-1]*Msun*Msun/(0.34*Rsun)


# plt.axhline(np.log10(E_available), color='k', linestyle='dotted', label="Available from orbit")
#plt.text("")

ax[0].set_xlabel('Star Age (yr)')
ax[0].set_ylabel('Cumulative Energy [erg]')
ax[0].set_xlim(0,20)


# plt.ylim(45, 48)


ax[0].legend(loc=0,fontsize=8)



# ax[1].plot(time,1-(E_rad+E_kin)/(E_CE+E_nuc))
ax[1].plot(time,(E_grav)/(E_CE))

ax[1].set_xlabel('Star Age (yr)')
ax[1].set_ylabel(r"Effective $\alpha_{CE}=\Delta E_{grav}/E_{inj.}$")
# ax[1].set_xlim(0,20)
ax[1].set_ylim(0,6)



plt.tight_layout()
plt.savefig("effective_alpha_BH.pdf")
plt.show()



In [ ]:
fig = plt.figure(figsize=(4,3), dpi=300)

secyer = 31622400.0



i=0
for run_name in run_names:
    lum = 10.0**modelsBH[i].history['log_L']
    time_step = 10.0**modelsBH[i].history['log_dt'] * secyer
    time = modelsBH[i].history["star_age"]
    E_CE = np.cumsum(time_step*modelsBH[i].history["CE_energy_rate"])
    E_rad = np.cumsum(lum*time_step)* 3.839e33
    E_nuc = np.cumsum(lum[0]*time_step)* 3.839e33
    E_grav = modelsBH[i].history["total_gravitational_energy"]-modelsBH[i].history["total_gravitational_energy"][0]
    E_grav = E_grav -np.min(E_grav)


    
#    E_kin = modelsBH[i].history["total_linear_kinetic_energy"]

    plt.plot(time,(E_grav)/(E_CE+E_nuc))


    plt.plot(modelsBH[i].history['star_age'], E_rad/(E_CE), linewidth=2, label=labels[i])
#     plt.plot(modelsBH[i].history['star_age'], 1-E_rad/(E_CE+E_nuc), linewidth=2, label=labels[i])
    i=i+1
#plt.plot(static_t,static_m,linewidth=2,linestyle="--",color="k")

#plt.legend(loc=0,fontsize=8)
plt.xlabel('Star Age (yr)')
plt.ylabel(r"Effective $\alpha_{CE}=1-\frac{E_{inj.}}{E_{rad.}+E_{kin.}}$")
plt.xlim(0,25)



plt.tight_layout()
plt.savefig("effective_alpha_BH_comparison.pdf")
plt.show()



In [ ]:
masses_TML = np.linspace(0.5,27.,54)
xvals_TML = np.linspace(1.1,1.9,54)


fig1 = plt.figure(figsize=(10,7.5))
ax1 = fig1.add_subplot(111)
#fig1.subplots_adjust(top=0.80, left=0.12, right=0.9, bottom=0.12)
a.SetParameters(cmap_dynamic_range=2.8, masses_TML=masses_TML, xvals_TML=xvals_TML,
               signed_log_cmap=False, Yaxis='log_radius', tau10=True, tau100=True, Xaxis="star_age",
               czones=False,cmap='RdBu_r', Variable='v_div_vesc')
a.Kippenhahn(ax1)
plt.savefig("v_div_vesc_BH.pdf")
plt.show()

fig1 = plt.figure(figsize=(10,7.5))
ax1 = fig1.add_subplot(111)
#fig1.subplots_adjust(top=0.80, left=0.12, right=0.9, bottom=0.12)
a.SetParameters(cmap_dynamic_range=6.0, signed_log_cmap=False, Variable='opacity')
a.Kippenhahn(ax1)
plt.savefig("opacity_BH.pdf")
plt.show()




fig1 = plt.figure(figsize=(10,7.5))
ax1 = fig1.add_subplot(111)
#fig1.subplots_adjust(top=0.80, left=0.12, right=0.9, bottom=0.12)
a.SetParameters(cmap_dynamic_range=2.0, signed_log_cmap=False, Variable='ionization_energy')
a.Kippenhahn(ax1)
plt.savefig("ionization_energy_BH.pdf")
plt.show()


fig1 = plt.figure(figsize=(10,7.5))
ax1 = fig1.add_subplot(111)
#fig1.subplots_adjust(top=0.80, left=0.12, right=0.9, bottom=0.12)
a.SetParameters(cmap_dynamic_range=10.0, signed_log_cmap=False, Variable='density')
a.Kippenhahn(ax1)
plt.savefig("density_BH.pdf")
plt.show()

fig1 = plt.figure(figsize=(10,7.5))
ax1 = fig1.add_subplot(111)
a.SetParameters(cmap_dynamic_range=3.0, signed_log_cmap=False, Variable='super_ad')
a.Kippenhahn(ax1)
plt.savefig("super_ad_BH.pdf")
plt.show()



fig1 = plt.figure(figsize=(10,7.5))
ax1 = fig1.add_subplot(111)
#fig1.subplots_adjust(top=0.80, left=0.12, right=0.9, bottom=0.12)
a.SetParameters(cmap_dynamic_range=2.0, signed_log_cmap=False, Variable='entropy', Yaxis='mass')
a.Kippenhahn(ax1)
plt.savefig("entropy_BH.pdf")
plt.show()

fig1 = plt.figure(figsize=(10,7.5))
ax1 = fig1.add_subplot(111)
#fig1.subplots_adjust(top=0.80, left=0.12, right=0.9, bottom=0.12)
a.SetParameters(cmap_dynamic_range=2.0, signed_log_cmap=False, Variable='temperature', Yaxis='mass', Xaxis="star_age")
a.Kippenhahn(ax1)
plt.savefig("vel_BH.pdf")
plt.show()






